In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import os

os.chdir("/content/gdrive/My Drive/kaggle/sentiment analysis/sentiment data")

In [ ]:
import pandas as pd

train = pd.read_csv('train.txt', header=None, sep=';')

train.columns = ['text', 'target']

val = pd.read_csv('val.txt', header=None, sep=';')

val.columns = ['text', 'target']

test = pd.read_csv('test.txt', header=None, sep=';')

test.columns = ['text', 'target']

In [ ]:
test.head()

,text,target
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [ ]:
train['target'].value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: target, dtype: int64

In [ ]:
y_val = val['target']

y_test = test['target']


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = train['target']

y_train = le.fit_transform(y_train)


from keras.utils import to_categorical

y_train = to_categorical(y_train, 6)


In [ ]:
y_val = le.transform(y_val)

y_test = le.transform(y_test)
y_test_2 = y_test
y_val = to_categorical(y_val, 6)
y_test = to_categorical(y_test, 6)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()
sw = set(stopwords.words('english'))
import re

In [ ]:
def clean_text(text):
  text = text.lower()
  
  text = re.sub('\[[^]]*\]', ' ', text)
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = text.split()
  
  text = [ps.stem(w) for w in text if w not in sw]
  
  text = ' '.join(text)
  return text



In [ ]:
text = train.text

In [ ]:
cleaned_text = text.apply(clean_text)

In [ ]:
cleaned_text

0                                        didnt feel humili
1        go feel hopeless damn hope around someon care ...
2                     im grab minut post feel greedi wrong
3           ever feel nostalg fireplac know still properti
4                                             feel grouchi
                               ...                        
15995        brief time beanbag said anna feel like beaten
15996     turn feel pathet still wait tabl sub teach degre
15997                              feel strong good overal
15998                       feel like rude comment im glad
15999                         know lot feel stupid portray
Name: text, Length: 16000, dtype: object

In [ ]:
val_text = val.text
cleaned_val_text = val_text.apply(clean_text)

test_text = test.text
cleaned_test_text = test_text.apply(clean_text)

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 4000)

tokenizer.fit_on_texts(cleaned_text)

cleaned_text = tokenizer.texts_to_sequences(cleaned_text)

cleaned_val_text = tokenizer.texts_to_sequences(cleaned_val_text)
cleaned_test_text = tokenizer.texts_to_sequences(cleaned_test_text)

In [ ]:
max=0

for i in range (0,len(cleaned_text)):
  if(len(cleaned_text[i])>max):
    max = len(cleaned_text[i])


In [ ]:
print(max)

35


In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
X_train = pad_sequences(cleaned_text, maxlen=35)
X_val = pad_sequences(cleaned_val_text, maxlen=35)
X_test = pad_sequences(cleaned_test_text, maxlen=35)

In [ ]:
from tensorflow import keras 


In [ ]:
model = keras.models.Sequential([
                                 keras.layers.Embedding(4000, 32, input_length=35),
                                 keras.layers.LSTM(128, return_sequences=True),
                                 keras.layers.GRU(128),
                                 keras.layers.Dense(6, activation='sigmoid')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 35, 32)            128000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 35, 128)           82432     
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 774       
Total params: 310,278
Trainable params: 310,278
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=5)

Epoch 1/5
500/500 [==============================] - 4s 9ms/step - loss: 1.2458 - accuracy: 0.4462 - val_loss: 0.8392 - val_accuracy: 0.6175
Epoch 2/5
500/500 [==============================] - 4s 8ms/step - loss: 0.4222 - accuracy: 0.8462 - val_loss: 0.3017 - val_accuracy: 0.8975
Epoch 3/5
500/500 [==============================] - 4s 8ms/step - loss: 0.1914 - accuracy: 0.9280 - val_loss: 0.2821 - val_accuracy: 0.9000
Epoch 4/5
500/500 [==============================] - 4s 8ms/step - loss: 0.1377 - accuracy: 0.9452 - val_loss: 0.2699 - val_accuracy: 0.9020
Epoch 5/5
500/500 [==============================] - 4s 8ms/step - loss: 0.1100 - accuracy: 0.9549 - val_loss: 0.2766 - val_accuracy: 0.9050


In [ ]:
import numpy as np
preds = model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [ ]:
count=0
for i in range(2000):
  if y_test_2[i] == preds[i]:
    count += 1

In [ ]:
print(count/2000)

0.8985
